In [ ]:
import pandas as pd
from urllib.parse import quote_plus, urlencode
from urllib.request import urlopen, Request
import json
import random

def jusoAPI(keystr):
    if '오산시' in keystr:
        return None
    if type(keystr) == str:
        key_list = list(keystr.split())
    else:
        return None
    for i in range(len(key_list), 3, -1):
        addr = ' '.join(key_list[:i])
        url = 'http://www.juso.go.kr/addrlink/addrLinkApi.do'
        queryParams = '?' + urlencode(
            {quote_plus('currentPage'): '1', quote_plus('countPerPage'): '6', quote_plus('resultType'): 'json',
            quote_plus('keyword'): addr, quote_plus('confmKey'): 'devU01TX0FVVEgyMDIxMTAwMjIzNDkxMzExMTcxNDg='})
        while True:
            try:
                request = Request(url + queryParams)
                request.get_method = lambda: 'GET'  # default GET anyway
                response_body = urlopen(request).read()
                break
            except:
                continue

        root_json = json.loads(response_body)
        total = int(root_json['results']['common']['totalCount'])
        if total == 0:
            return None
        elif total == 1:
            ad = root_json['results']['juso'][0]
            return ad['roadAddr']  # ['detBdNmList']
        else:
            num = total if total < 6 else 6
            ad = root_json['results']['juso'][random.randint(0, num-1)]
            return ad['roadAddr']


def date_change(date):
    new_date = date[:4] + date[5:7] + date[8:]
    return int(new_date)


df_all = pd.read_csv('./df_all.csv')
df = df_all.drop(['Unnamed: 0', 'Unnamed: 0.1', '구분명', '집화일자', '집배일자', '운임명', '집화여부', '집배일자', '장비구분', '품목'], axis=1)
df = df[['배달일자', 'SM명', '수량(BOX)', '운임', '집배시간', '받는분주소']]
df = df.dropna(axis = 0)
df.배달일자 = df.배달일자.apply(date_change)
df_sorted = df.sort_values(by=['배달일자', 'SM명'], ascending=True)
postman = len(df_sorted[df_sorted.배달일자 == 20210302].SM명.value_counts().unique())
df_210302 = df_sorted[df_sorted.배달일자 == 20210302]
df_210302.loc[:, '받는분주소'] = df_210302.loc[:, '받는분주소'].apply(jusoAPI)
df_210302 = df_210302.dropna(axis=0)
df_210302.to_csv(f"20210302_{postman}명4.csv")

In [ ]:
df_new = df_210302.sample(n=300, random_state=45)
df_new.drop('Unnamed: 0',axis=1, inplace=True)
df_new = df_new.loc[:, '받는분주소']
df_new.reset_index(drop=True, inplace=True)
df_new.to_csv("addr300.csv")